[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/twMr7/Python-Tutorial/blob/master/13-Pandas_Data_Analysis.ipynb)

# 13. Pandas 資料分析

[Pandas](http://pandas.pydata.org/) 也是建構於 Numpy 之上，主要設計的定位是用來資料處理及分析。 與 Numpy 陣列不同的地方在，Pandas 表格式的資料容器 `DataFrame` 可以存放及操作異質資料型態，資料欄位可以有標籤，易於處理 missing data、類別資料、與時間序列資料，而且針對常用的資料儲存格式提供了相當廣泛的輸出入的支援。

一下教材內容節錄自 [Pandas 官方文件](http://pandas.pydata.org/pandas-docs/stable/index.html)。

+ [**13.1 Series 與 DataFrame 基本認識**](#basic-datatype)
+ [**13.2 資料內容選取**](#indexing-selecting)
+ [**13.3 新增、刪除與合併**](#append-concat)
+ [**13.4 深入檢視**](#inspecting-data)
+ [**13.5 分群及排序**](#grouping-sorting)
+ [**13.6 漏失數據處理**](#missing-data)
+ [**13.7 時間序列**](#time-series)

### § 使用 `pandas` 套件

In [ ]:
import numpy as np
import pandas as pd

<a id="basic-datatype"></a>

## 13.1 `Series` 與 `DataFrame` 基本認識

Pandas 主要的資料結構是 `Series` 與 `DataFrame`。
+ `Series` - 一維，有標籤，同質性（homogeneously-typed）的資料結構。
+ `DataFrame` - 二維，有欄位標籤及列記錄標籤，欄位異質性（heterogeneously-typed ）的資料結構。

標籤就像 Dict 容器的 Key，可以是字串、數值、時間等可以當成 Key 的資料型態。 Series 可以想成是每一個元素位置都帶有標籤的 **row向量** 或 **column向量**，由 DataFrame 取出某個 row 或某個 column 就是一個 Series，取出的 Series 就帶有原本的 **row標籤** 或 **column標籤**。 

在建立 DataFrame 或 Series 時，若沒有指定標籤，預設會使用位置順序的數值序號。

In [ ]:
# Creating a Series by passing a list of values, letting pandas create a default integer index
pd.Series([1, 3, 5, np.nan, 6, 8])

In [ ]:
# Creating a DataFrame by passing a NumPy array, with a datetime index and labeled columns
dates = pd.date_range('20190401', periods=6)
print(dates)

In [ ]:
df = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
df

In [ ]:
# Creating a DataFrame by passing a dict of objects that can be converted to series-like.
# 注意： 單一值會自動 broadcast
df2 = pd.DataFrame({'A': 1.,
                    'B': pd.Timestamp('20130102'),
                    'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                    'D': np.array([3] * 4, dtype='int32'),
                    'E': pd.Categorical(["test", "train", "test", "train"]),
                    'F': 'foo'})
df2

### § 基本數據檢視


In [ ]:
# 檢視前幾筆
df.head(3)

In [ ]:
# 檢視後幾筆
df.tail(3)

In [ ]:
# 檢視記錄標籤
df.index

In [ ]:
# 檢視記錄欄位標籤
df.columns

In [ ]:
# 檢視資料類型
df.dtypes

In [ ]:
# 資料都是數值的話，轉成 numpy 陣列非常快
# Pandas 版本 < 0.24 要使用 DataFrame.values
df.to_numpy()

In [ ]:
# 檢視資料類型
df2.dtypes

In [ ]:
# 異質欄位資料也可以轉成 numpy 陣列的話，但成本較高，通常在 numpy 也不會比較容易處理異質資料
df2.to_numpy()

In [ ]:
# 基本統計描述（column-wise）：平均值、標準差、最小值、第一四分位數、中位數、第二四分位數、最大值
df.describe()

In [ ]:
# Transpose
df.T

In [ ]:
# 根據某欄位排序
df.sort_values(by='B', ascending=False)

<a id="indexing-selecting"></a>

##  13.2 資料內容選取

### § 直接括號操作

注意： Pandas 可以直接用中括號 [  ] 選取 DataFrame 的資料，由於括號中的標籤可以是欄位標籤也可以是列序號，在容易混淆誤用時，請儘可能使用下一節中提到的 `loc[]` 或 `iloc[]` 語法。

DataFrame 可以直接使用中括號 [  ] 指定欄位標籤存取欄位資料，主要語法為：
```
    DataFrame[欄位標籤], 或
    DataFrame[欄位標籤清單]
```
若只需要存取單一欄位，而且 columns 標籤是字串形式，也可以使用以下語法（注意不帶字串的引號）：
```
    DataFrame.欄位標籤
```

Series 直接使用中括號 [  ] 的語法為：
```
    Series[標籤], 或
    Series[標籤清單]
```
若只需要存取序列的單一位置，而且 index 標籤是字串形式，也可以使用以下語法（注意不帶字串的引號）：
```
    Series.index標籤
```

In [ ]:
# 選取單一欄位，返回一個 Series
df['A']

In [ ]:
# df['A'] 語法等同於 df.A
df.A

In [ ]:
# 資料型別轉換
df.A.astype(int)

### § 使用 `loc` 及 `iloc` 操作

建議使用針對 Pandas 資料結構最佳化過的 `loc[]`、和 `iloc[]` 語法，序號（indexing）及片段（slicing）的語法類似 Numpy 陣列。

| `loc` 存取方式   |  Series            | DataFrame                     |
|------------------|--------------------|-------------------------------|
|   標籤           | `s.loc[標籤]`      | `df.loc[標籤, 標籤]`          |
|   標籤清單       | `s.loc[標籤清單]`  | `df.loc[標籤清單, 標籤清單]`  |
|   標籤片段       | `s.loc[標籤片段]`  | `df.loc[標籤片段, 標籤片段]`  |
| Boolean 遮罩陣列 | `s.loc[遮罩陣列]`  | `df.loc[遮罩陣列, 遮罩陣列]`  |

注意片段語法 `loc[start:stop:step]`，返回結果為封閉區間的 [start, stop]，**包含**結束的 stop 項。

`iloc` 主要以位置順序的數值序號為存取方式。

| `iloc` 存取方式  |  Series            | DataFrame                     |
|------------------|--------------------|-------------------------------|
|   序號           | `s.iloc[序號]`     | `df.iloc[序號, 序號]`         |
|   序號清單       | `s.iloc[序號清單]` | `df.loc[序號清單, 序號清單]`  |
|   序號片段       | `s.iloc[序號片段]` | `df.loc[序號片段, 序號片段]`  |

注意片段語法 `iloc[start:stop:step]`，返回結果為半開放區間的 [start, stop)，**不包含**結束的 stop 項。


In [ ]:
# 使用 loc，選取特定列標籤及欄位標籤位置
df.loc['2019-04-02', 'A']

In [ ]:
# 使用 loc，選取特定列標籤，欄位標籤省略則預設為全選
df.loc['2019-04-02']

In [ ]:
# 使用 loc 選取特定列後，再指定欄位標籤
df.loc['2019-04-02'].D

In [ ]:
# 所有列（slicing 語法）及部分欄位選取，注意標籤可重複
df.loc[:, ['A', 'C', 'A']]

In [ ]:
# 部份列片段，及部分欄位片段選取，注意片段有包含 stop 項
df.loc['20190402':'20190404', 'A':'C']

In [ ]:
# 使用 iloc，序號選取特定列及欄位
df.iloc[3, 1]

In [ ]:
# 使用 iloc，選取特定列序號，欄位序號省略則預設為全選
df.iloc[3]

In [ ]:
# 使用 iloc 選取特定列後，再指定欄位標籤
df.iloc[3].C

In [ ]:
# 類似 numpy 的序號與片段語法，注意片段沒有包含 stop 項
df.iloc[3:5, 0:2]

###  §  使用比較運算產生遮罩

+ **比較運算子** - 運算結果返回 `bool` 陣列。

| 比較運算操作           | 說明          |
|------------------------|---------------|
| **X < Y**              | 小於          |
| **X <= Y**             | 小於或等於    |
| **X > Y**              | 大於          |
| **X >= Y**             | 大於或等於    |
| **X == Y**             | 等於          |
| **X != Y**             | 不等於        |
| **(條件1) & (條件2)**  | 真值邏輯 AND  |
| **(條件1) \| (條件2)** | 真值邏輯 OR   |
| **~(條件1)**           | 真值邏輯 NOT  |


In [ ]:
# 類似 Numpy，比較結果為 Boolean 遮罩
df.A > 0

In [ ]:
# Boolean 陣列選取，使用單一欄位的條件
df.loc[df.A > 0]

In [ ]:
# 兩個條件的邏輯 OR 比較，要用 | 符號
(df.A > 0) | (df.B > 0)

In [ ]:
df.loc[(df.A > 0) | (df.B > 0)]

<a id="append-concat"></a>

## 13.3 新增、刪除與合併

DataFrame 可以直接用括號 [  ] 指定新標籤來新增欄位，也可以用 `loc` 語法指定新的標籤來新增列或欄位數據。 另外也可以使用物件方法及 concat 函式：
+ `DataFrame.append()` - 新增列數據在最後，返回新物件。
+ `DataFrame.assign()` - 新增新欄位，返回新物件。
+ `Series.append()` - 串接另一個 Series，返回新物件。
+ `Pandas.concat()` - 串接 Pandas 的 Series 或 DataFrame 物件。

刪除使用 drop 物件方法：
+ `DataFrame.drop()` - 刪除指定的行或列標籤，可就地變更。
+ `Series.drop()` - 刪除指定標籤，可就地變更。


In [ ]:
# 追加一個欄位資料
df['E'] = ['one', 'one', 'two', 'three', 'four', 'three']
df

In [ ]:
# 使用 loc 新增列數據
df.loc[pd.to_datetime('2019-04-07'),:] = pd.Series({'A':0.1, 'B':0.2, 'C':0.3, 'D':0.4, 'E':'four'})
df

In [ ]:
# 使用 loc 給新欄位標籤新增欄位數據
df.loc[:, 'F'] = [True, False, True, True, False, True, False]
df

In [ ]:
# 使用 Series.isin 返回 Boolean 遮罩
df.E.isin(['two', 'four'])

In [ ]:
# 使用 Series.isin 返回的遮罩選取
df.loc[df.E.isin(['two', 'four'])]

In [ ]:
# 串接，預設 axis 0
pd.concat([df.iloc[[1,3]], df.loc[df.E.isin(['two', 'four'])]])

In [ ]:
# 用 append 追加數據列
s3 = df.iloc[3]
s3.name = pd.to_datetime('2019-04-07')

df.append(s3)

In [ ]:
# 刪除欄位，可就地刪除
df.drop(columns=['E','F'], inplace=False)

<a id="inspecting-data"></a>

## 13.4 深入檢視

時常取得的數據資料只有簡短的文字說明，就算有通常也很難完整交代數據的細節。 除了用 `DataFrame.decribe()` 整體檢視外，個別欄位或列 Series 也有敘述統計指標的方法可以用，另外也還有許多常用的方法可以用來深入檢視：
+ 重複或類別數據 - `nunique()`、`unique()`、`value_counts()`、`duplicated()`、`drop_duplicates()`、`equals()`。
+ 極值關係 - `idxmax()`、`idxmin()`。
+ 轉換 - `apply()`、`map()`、`transform()`。


In [ ]:
# 檢視某欄位的中位數
df.D.median()

In [ ]:
# 各欄位有多少不重複的值
df.nunique(axis=0)

In [ ]:
# 不重複的值是那些（類別）
df.E.unique()

#df.E.drop_duplicates()

In [ ]:
# 各類別出現次數
df.E.value_counts()

In [ ]:
# 某種數據比例條件下，最大/最小比值是 E 欄位的那一種類別
df.loc[(df.A / df.D).idxmax(), 'E']

In [ ]:
# 根據數值意義取門檻值，轉成三種類別
cut3levels = lambda row: 'high' if row.A > 1 else ('middle' if row.A > 0 else 'low')
df.apply(cut3levels, axis='columns')

In [ ]:
# 計算符合特定條件的數據出現次數
negcount = lambda x: x < 0
n_negA = df.A.map(negcount).sum()
n_negB = df.B.map(negcount).sum()
pd.Series([n_negA, n_negB], index=['# Negatives in A', '# Negatives in B'])

<a id="grouping-sorting"></a>

## 13.5 分群及排序

DataFrame 及 Series 都有一個 `groupby()` 方法，會根據指定標籤分群返回一個特別的 [`GroupBy`](http://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) 類別，這個類別會在另外套用方法後產生實際分群計算的結果（類似生成函式的概念）。 `GroupBy` 物件也可以用在迴圈中迭代，針對每群個別處理。 `groupby()` 函式的第一個參數 `by` 是分群的依據，也是分群結果的 key index。`by` 參數可以是：
+ 欄位標籤，或欄位標籤清單
+ DataFrame[欄位標籤]，或 DataFrame[欄位標籤清單]
+ Series - 由 DataFrame 選取產生的結果


In [ ]:
# 根據欄位 E 分群，群中各有多少
df.groupby('E').size()

In [ ]:
# 根據欄位 E 分群，計算每群中的 F 欄位有多少 True
df.groupby('E').F.sum()

In [ ]:
# 根據 F 欄分群，輸出分群結果
for name, group in df.groupby('F'):
    print('Group', name, ':')
    print(group, '\n')

In [ ]:
# 根據 F 欄分群，各群中最大的 C 欄值是多少？
df.groupby('F').C.max()

In [ ]:
# 根據 F 欄分群，各群中數值欄位的平均值是多少？
df.groupby('F').mean()

In [ ]:
# 根據 A 欄四捨五入的整數分群並排序，其他各欄位的平均值
df.groupby(df.A.round())['B', 'C', 'D'].mean().sort_index(ascending=True)

In [ ]:
# 根據 A 欄正負分群，其他各欄位平均值
cut2levels = lambda row: 'A+' if row.A > 0 else 'A-'
df.groupby(df.apply(cut2levels, axis='columns'))['B', 'C', 'D'].mean()

### § 聚合處理 Aggregation

Pandas 的 `aggregate()`（別名： `agg()`）用來套用函式作整體的向量式運算，與 Numpy 函式不同的是，Pandas 的 `aggregate` 永遠只會針對欄位方向或列方向套用。 `aggregate()` 函式接受的參數為：
+ 函式，或函式清單
+ 函式名稱字串，或函式名稱字串的清單
+ 以標籤當 Key 的字典，對應的值可以是上述的可接受參數

內建可以直接使用字串名稱的函式： `sum`、`mean`、`min`、`max`、`size`、`count`、`std`、`var`、`sem`（standard error of the mean）、`describe`。

In [ ]:
# 根據 A 欄正負分群，B 欄的最大及最小值
df.groupby(df.apply(cut2levels, axis='columns')).B.agg(['min','max'])

In [ ]:
# 根據 F 欄分群，各欄位的最大及最小值
df.groupby('F').agg({'A':['min','max'], 'B':['min','max'], 'C':['min','max'], 'D':['min','max']})

In [ ]:
# 根據 E 欄分群，C 欄位的最大及最小值，並以最小值排序
df.groupby('E').C.agg(['min','max']).sort_values(by=['min'], ascending=False)

<a id="missing-data"></a>

## 13.6 漏失數據處理

漏失數據（missing data，又稱 **NA**）在 Pandas 中主要使用 `numpy.nan`（Not a Number）形態表示，但也不排除使用 Python 的 `None` 來指定。 要注意的是，兩個 `numpy.nan` 互相比較永遠不會相等，但是 `None` 會相等，所以偵測漏失數據要使用 Pandas 提供的 `isna()` 函式。

In [ ]:
# NaN 永遠不會等於 NaN
print('(NaN == NaN) is', np.nan == np.nan)
# None 等於 None
print('(None == None) is', None == None)

In [ ]:
# 刪除欄位，可就地刪除
df.drop(columns=['E'], inplace=True)
# 刻意製造含 NaN 的數據，不常用這樣的選取方式
dfmiss = df[df > 0]
dfmiss

In [ ]:
# 偵測 NA 返回 Boolean 遮罩，注意：不能使用 dfmiss == np.nan 這樣的比較
dfmiss.isna()

In [ ]:
# 計算各欄位分別有多少漏失數據
dfmiss.isna().sum()

In [ ]:
# 針對 datetime 類型資料，Pandas 內部另外提供了 `NaT` 的資料類型來代表漏失的時間數據。
dfmiss['T'] = pd.Timestamp('20190417')
print(dfmiss)

In [ ]:
dfmiss.iloc[[1, 3, 4, 5], 5] = None
print(dfmiss)

In [ ]:
# 敘述統計函式，sum 把 NA 當 0，mean, cunsum 掠過
print('column A sum =', dfmiss['A'].sum())
print('column A mean =', dfmiss['A'].mean())
print('column A cumsum =\n', dfmiss['A'].cumsum())

In [ ]:
# 把有漏失任何欄位值的記錄丟掉，也可指定全部欄位沒有值才丟
dfmiss.dropna()

In [ ]:
# 丟掉很可惜，填補值來用
print('【填補前】：\n{}\n'.format(dfmiss))

print('【填補 0】：\n{}\n'.format(dfmiss.fillna(0)))

print('【後向填補】：\n{}\n'.format(dfmiss.fillna(method='bfill')))

print('【前向填補】：\n{}\n'.format(dfmiss.fillna(method='ffill')))

In [ ]:
# 使用內插值填補，部份方法來自 scipy.interpolate 模組
print('【填補前】：\n{}\n'.format(dfmiss))

print('【linear 內插填補】：\n{}\n'.format(dfmiss.interpolate(method='linear')))

print('【pchip 內插填補】：\n{}\n'.format(dfmiss.interpolate(method='pchip')))

print('【krogh 內插填補】：\n{}\n'.format(dfmiss.interpolate(method='krogh')))

<a id="time-series"></a>

## 13.7 時間序列

Pandas 在 NumPy 的 `datetime64` 和 `timedelta64` 資料形態的基礎上，建構了相當多針對時間序列作處理的功能。 主要以四種資料類型來處理不同的時間概念：
+ [`Timestamp`](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html) - (**Date times**時間概念) 支援時區資訊的特定日期時間，類似 Python 標準函式庫裡的 `datetime.datetime`。
+ [`Timedelta`](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timedelta.html) - (**Time deltas**時間概念) 絕對的連續性時間間距。類似 Python 標準函式庫裡的 `datetime.timedelta`。
+ [`Period`](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Period.html) - (**Time spans**時間概念) 週期性時間間隔的時間點。
+ [`DateOffset`](http://pandas.pydata.org/pandas-docs/stable/reference/offset_frequency.html) - (**Date offsets**時間概念) 用於不同曆法的相對時間間距。

相對於時間概念的操作，Pandas 提供的資料形態：

| 時間概念           | 純量類別     |   陣列類別       | Pandas 資料類別   | 主要建立方法                            |
|--------------------|--------------|------------------|-------------------|-----------------------------------------|
| **Date times**     | `Timestamp`  | `DatetimeIndex`  | `datetime64[ns]`  | `to_datetime()` 或 `date_range()`       |
| **Time deltas**    | `Timedelta`  | `TimedeltaIndex` | `timedelta64[ns]` | `to_timedelta()` 或 `timedelta_range()` |
| **Time spans**     | `Period`     | `PeriodIndex`    | `period[freq]`    | `Period()` 或 `period_range()`          |
| **Date offsets**   | `DateOffset` | None             | None              | `DateOffset()`                          |


In [ ]:
# Date time 資料型態的建立可以接受各種不同的格式
import datetime
pd.to_datetime(['4/1/2019', np.datetime64('2019-04-01'), datetime.datetime(2018, 4, 1)])

```
    date_range(start, end, periods, freq, ...)

    參數：
        start - 開始時間
        end - 結束時間
        periods - 共產生幾個時間點
        freq - 指定時距週期頻率的格式字串
```

[`date_range`](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html) 的 `freq` 參數可接受指定格式字串：
+ `D` - calendar day frequency
+ `B` - business day frequency
+ `W` - weekly frequency
+ `M` - month end frequency
+ `Q` - quarter end frequency
+ `Y` - year end frequency
+ `H` - hourly frequency
+ `min` - minutely frequency
+ `S` - secondly frequency
+ `ms` - milliseconds
+ `us` - microseconds
+ `N` - nanoseconds


In [ ]:
# 產生固定時間週期的序列
pd.date_range('2019-04-01', periods=6, freq='H')

In [ ]:
# Timestamp 物件的建立
aFriday = pd.Timestamp('2019-05-03')
print(aFriday.date(), 'is', aFriday.day_name())

In [ ]:
# 增加一天（絕對的連續性時間間距）
aSaturday = aFriday + pd.Timedelta('1 day')
print(aFriday.date(),'加一天 =', aSaturday.date(), 'is', aSaturday.day_name())

In [ ]:
# 增加一個工作天（Business Day，相對的時間間距概念）
aMonday = aFriday + pd.offsets.BDay()
print(aFriday.date(),'加一個工作天 =', aMonday.date(), 'is', aMonday.day_name())

In [ ]:
# 時間序列的 DataFrame 及 Series 資料，可以使用 Timestamp 當作 index
pd.Series(np.random.randn(3), index=[pd.Timestamp('2019-05-01'), pd.Timestamp('2019-05-02'), pd.Timestamp('2019-05-03')])

In [ ]:
# 時間序列的 DataFrame 及 Series 資料，也可以使用 Period 當作 index
pd.Series(np.random.randn(3), index=[pd.Period('2019-04'), pd.Period('2019-05'), pd.Period('2019-06')])

In [ ]:
# 如果日期時間被分割成不同的欄位，也可以用 to_datetime() 作合併轉換，但欄位標籤要是可以辨識的名字，如： year, month, day, ...
dfYMDH = pd.DataFrame({'year': [2015, 2016], 'month': [2, 3], 'day': [4, 5], 'hour': [2, 3]})
print('時間欄位被切割的 DataFrame:\n', dfYMDH)

pd.to_datetime(dfYMDH)

In [ ]:
# Series 的 index 是時間，可以用中括號及時間字串選取資料
df.A['2019-04-02':'2019-04-06']

In [ ]:
# 可以用部分字串選取大範圍時間
df.B['2019-04']

In [ ]:
# 使用 shift() 將所有資料列向前（正的 Period）或向後（負的 Period）移動，
# 注意： 試試 df.shift(1)，沒有指定 freq 參數的話，資料與時間不會對齊
df.shift(1, freq='D')

In [ ]:
# 使用 tshift() 將所有資料列向前（正的 Period）或向後（負的 Period）移動，
# 結果與 shift(1, freq='D') 一樣
df.tshift(1)

### § 時間序列重新取樣

針對 index 是時間序列的 Series 和 DataFrame，Pandas 設計了有如**“時間 groupby”**的 [**resampler**](http://pandas.pydata.org/pandas-docs/stable/reference/resampling.html) 機制，容許簡單有效率地對時間序列作頻率的轉換。 Series 和 DataFrame 分別都提供了方法用來建立 **resampler** 物件：
+ [`Series.resample()`](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.resample.html)
+ [`DataFrame.resample()`](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html)


In [ ]:
# Downsampling 成兩天一群
df.resample('2D').indices

In [ ]:
# Upsampling 成每6小時一群，原本沒有的資料如同漏失資料一樣要再進行填補
df.resample('6H').interpolate().fillna(method='ffill')